In [98]:
# initializing

import pandas as pd
import pandas_gbq as pgbq
import multiprocessing
from scipy import stats

network_type = 'transaction'
sql_modules = ['edges','edge_weight','indegree','outdegree']
google_project_id = 'solid-ridge-104914'


#if community facts are already saved, use those otherwise re-calculate and save

try:
    container = pgbq.read_gbq('select * from community_networks.community_facts', google_project_id, dialect='standard')
except:
    print("Community facts table does not exist, generating now.")
    with open('../sql/cumulative_networks/community_facts.txt') as query_file:
        query = query_file.read()
    container = pgbq.read_gbq(query, google_project_id, dialect='standard')
    print("Saving result to new table")
    container.to_gbq(result, 'community_networks.community_facts', google_project_id, if_exists='replace')


Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 2737 rows.

Total time taken 3.49 s.
Finished at 2017-07-10 18:03:53.


In [93]:
# worker function

def bigquery_worker(i):
    sql_module = sql_modules[i]
    print('trying module ',i,': ',sql_modules[i], sep='')
    try:
        result = pgbq.read_gbq('select * from community_networks.'+network_type+'_'+sql_module, google_project_id, dialect='standard')
        print(network_type,'_',sql_module,' table found.', sep='')
    except:
        print(network_type,'_',sql_module,' table does not exist, generating now')
        with open('../sql/cumulative_networks/'+network_type+'/'+sql_module+'.txt') as query_file:
            query = query_file.read()
        result = pgbq.read_gbq(query, google_project_id, dialect='standard')
        print('Creating new ',network_type,'_',sql_module,' table')
        pgbq.to_gbq(result, 'community_networks.'+network_type+'_'+sql_module, google_project_id, if_exists='replace')
    return (result)

In [99]:
#make it work

if __name__ == '__main__':
    with Pool(processes=len(sql_modules)) as pool:
        result = pool.map(bigquery_worker, (range(len(sql_modules),)))
        
for i in range(len(result)):
    container = container.join(result[i].set_index('cid'))

trying module  1 :  edge_weight
trying module  3 :  outdegree
trying module  2 :  indegree
trying module  0 :  edges
Requesting query... Requesting query... Requesting query... Requesting query... ok.
Query running...
ok.
Query running...
ok.
Query running...
ok.
Query running...
Query done.
Processed: 0.0 B

Query done.
Processed: 0.0 B

Retrieving results...
Got 2692 rows.

Retrieving results...
Got 2692 rows.

Total time taken 3.03 s.
Finished at 2017-07-10 18:04:02.
transaction _ edge_weight  table found.
Total time taken 2.75 s.
Finished at 2017-07-10 18:04:02.
transaction _ indegree  table found.
Query done.
Processed: 0.0 B

Retrieving results...
Got 2692 rows.

Total time taken 3.37 s.
Finished at 2017-07-10 18:04:02.
transaction _ outdegree  table found.
Query done.
Processed: 42.1 KB

Retrieving results...
  Got page: 1; 100% done. Elapsed 7.07 s.
Got 2692 rows.

Total time taken 7.12 s.
Finished at 2017-07-10 18:04:06.
transaction _ edges  table found.


In [ ]:
#Calculations

  'transaction' as edge_type,
  cf.*,
  avg_weight,
  if(nodes*(nodes-1)/2=0, 0, edges/(nodes*(nodes-1)/2))*100 as network_density_X_100,
  2*edges/nodes as avg_degree,
  avg_indegree,
  avg_outdegree,
  (avg_indegree - avg_outdegree) / avg_outdegree indegree_skew

In [61]:
from multiprocessing import Pool
import time

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(processes=4) as pool:         # start 4 worker processes

        it = pool.imap(f, range(10))
        print(next(it))                     # prints "0"
          # prints "4" unless your computer is *very* slow

        

0
1
